# Analytical Variance Based Decomposition

This is the calculation of the analytical variance based decomposition values. The math is not explained in detail and is only included for reference.

It required SymPy to run

In [ ]:
from sympy import *
init_printing()

In [ ]:
x,y,z,a,b = symbols('x y z a b')
F = sin(x) + a*sin(y)**2 + b*z**4*sin(x)
F

In [ ]:
def inner(F,*xyz):
    """Compute the inner product over the dims"""
    return simplify(integrate(F/((2*pi)**len(xyz)),*[(p,-pi,pi) for p in xyz]))

# Sobol Decomp

This isn't really needed but is worth the demonstration

In [ ]:
F0 = inner(F,x,y,z)
F0

In [ ]:
Fx = inner(F,y,z) - F0
Fx

In [ ]:
Fy = inner(F,x,z) - F0
Fy

In [ ]:
Fz = inner(F,x,y) - F0
Fz

In [ ]:
Fxy = inner(F,z) - Fx - Fy - F0
Fxy

In [ ]:
Fyz = inner(F,x) - Fy - Fz - F0
Fyz

In [ ]:
Fzx = inner(F,y) - Fz - Fx - F0
Fzx

In [ ]:
Fxyz = F - Fxy - Fyz - Fzx - Fx - Fy - Fz - F0
Fxyz

In [ ]:
F0 + Fx + Fy + Fz + Fxy + Fyz + Fzx + Fxyz

In [ ]:
def Var(FF,*xyz):
    return simplify(inner(FF*FF- inner(FF,*xyz)**2,*xyz))

In [ ]:
from collections import OrderedDict
xy = x*y;yz=y*z;zx=z*x;xyz=x*y*z
Vars = OrderedDict([(0,  Var(F0,x,y,z)),
                    (x,  Var(Fx,x,y,z)),
                    (y,  Var(Fy,x,y,z)),
                    (z,  Var(Fz,x,y,z)),
                    (xy, Var(Fxy,x,y,z)),
                    (yz, Var(Fyz,x,y,z)),
                    (zx, Var(Fzx,x,y,z)),
                    (xyz,Var(Fxyz,x,y,z))])
Vars

In [ ]:
VF = Var(F,x,y,z)
VarFrac = OrderedDict([(p,v/VF) for p,v in Vars.items()])
VarFrac

In [ ]:
for _ in VarFrac.items():
    print(_)

In [ ]:
def Eval(F):
    return F.evalf(subs={a:7.0,b:Rational(1,10)})

In [ ]:
for p,v in VarFrac.items():
    print(p,Eval(v))

# Alternative

Directly compute the integral

Note 

$$E[f|x_i] = \int_{x_{i\sim}}f(x)dx_{i\sim}$$

and

$$S_{x_i} = \frac{V[ E[f|x_i] ]}{V[f]} = \frac{E[E[f|x_i]^2] - E[E[F|x_i]]^2}{V[f]}$$

and

$$T_{x_i} = 1 - \frac{V[E[f|x_{i\sim}]]}{V[f]}$$

In [ ]:
def Egiven(F,*xyz):
    """ E[F,x] or E given some value. You integrate over the others"""
    xyz = {x,y,z} - set(xyz)
    return inner(F,*xyz)

def S(F,*xyz):
    return Var(Egiven(F,*xyz),x,y,z)/Var(F,x,y,z)
def T(F,*xyz):
    xyz = {x,y,z} - set(xyz)
    return 1 - S(F,*xyz)

In [ ]:
Ss = [(xyz,S(F,xyz)) for xyz in [x,y,z]]
Ss

In [ ]:
[(xyz,Eval(s)) for xyz,s in Ss]

In [ ]:
Ts = [(xyz,T(F,xyz)) for xyz in [x,y,z]]
Ts

In [ ]:
[(xyz,Eval(t)) for xyz,t in Ts]

In [ ]:
for s in Ss:
    dd = list(s) + [Eval(s[1])]
    print(dd)
for t in Ts:
    dd = list(t) + [Eval(t[1])]
    print(dd)